In [138]:
import pandas as pd
from datetime import datetime

# Crawl data from yahoo finance

In [139]:
def getCrumb(ticker):
    URL  = "https://finance.yahoo.com/quote/%s/history" % ticker
    import requests 
    import re
    r = requests.get(url = URL) 
    content = str(r.content)
    r1 = re.findall(r"CrumbStore\":\{\"crumb\":\"[A-Za-z0-9.\\\-]+\"",content)[0]
    crumb = r1.replace("CrumbStore\":{\"crumb\":\"","").replace('"',"")
    return crumb,r.cookies


def get_data(symbol, data_type, start_date , end_date):
    import requests 
    import re
    from io import StringIO
    import os
    import os.path
    
    directory = "data"
    filename = directory+"/"+symbol+"_"+data_type
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    #check cache
    last_saved = 86401
    if os.path.isfile(filename):
        statbuf = os.stat(filename)
        last_saved = datetime.now().timestamp() - statbuf.st_mtime
         
    if(last_saved>(86400)):
        #crawl
        api_url = "https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2={2}&interval={3}&events={4}&crumb={5}"
        (crumb, cookies)=getCrumb(symbol)
        api_url = api_url.format(symbol,0,datetime.now().strftime('%s'),"1d", data_type, crumb)
        r = requests.get(url = api_url, cookies = cookies.get_dict(".yahoo.com"))  
        content = str(r.content).replace(",",'","').replace("\\n",'"\r\n"').replace("b\'","").replace("'","")
        res = '"'+content+'"'
        
        #save
        text_file = open(filename, "w")
        n = text_file.write(r.content.decode('utf-8'))
        text_file.close()
    
    #load
    df = pd.read_csv(filename)
    df["Date"]=pd.to_datetime(df['Date'])
    df = df[df["Date"]>=start_date][df["Date"]<=end_date]
    return df  

def get_price_data(stock, start_date, end_date):
    print("Getting price data ...")
    price = get_data(stock,"history", start_date, end_date)
    price.rename(columns={'Date':'date','Open':'open','High':'high',
                          'Low':'low','Close':'close','Adj Close':'adjClose','Volume':'vol'}, inplace=True)
    price["date"]=pd.to_datetime(price['date'])
    price = price.set_index(["date"])
    return price
def get_dividend_data(stock, start_date, end_date):
    print("Getting dividend data ...")
    dividend = get_data(stock,"div", start_date, end_date)
    dividend.rename(columns={'Date':'date','Dividends':'dividend'}, inplace=True)
    dividend["date"]=pd.to_datetime(dividend['date'])
    dividend = dividend.set_index(["date"])
    return dividend

# Calculate returns

In [149]:
from datetime import timedelta, date
def calculate_profit(price, dividend, init_amt = 10000, fee = 0, monthly_topup = 0):
    print("Calculating returns ...")
    data = pd.DataFrame([]) 
    price = price.dropna()
    dividend = dividend.dropna()
    init_close = -1
    total_spend = init_amt
    amount = init_amt
    shares = 0
    curDate = price.index.min()
    last_close = 0
    last_open = 0
    pre_date = None
    while True:
        curDiv = 0
        if(curDate in price.index):
            if(pre_date is not None and pre_date.month != curDate.month):
                amount = amount + monthly_topup
                total_spend = total_spend + monthly_topup
            pre_date = curDate
            curRec = price.loc[[curDate]]
            curOpen = float(curRec["open"].values[0])
            curClose = float(curRec["close"].values[0])
            last_open = curOpen
            last_close = curClose
            if(init_close==-1):
                init_close = last_close
            if(amount>0):
                #buy shares
                share_can_buy = (int)(amount/(curOpen*(1+fee)))
                amount = amount - share_can_buy*(curOpen*(1+fee))
                shares = shares + share_can_buy
        if(curDate in dividend.index):
            curDiv = dividend.loc[[curDate]]["dividend"].values[0]
            amount = amount + (shares*float(curDiv))
        if(curDate == price.index.max()):
            break
        gross = shares*curClose + amount
        data = data.append({'Date' : curDate ,
                            'Spent': round(total_spend,2),
                            'Gross' : round(gross,2),
                            'Dividend': curDiv,
                            'Close': curClose,
                            'ClosePct': round(((curClose-init_close)/init_close)*100,2),
                            'NetPct' :  round(((gross-total_spend)/total_spend)*100,2),
                           } , ignore_index=True)
        curDate = curDate + timedelta(days=1)
    return data

# Plot line chart

In [185]:
def plot_chart(data, height, width):
    print("Ploting chart ...")
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    df = data
    df = df.sort_index(ascending=True)
    fig = go.FigureWidget(make_subplots(rows=1, specs=[[{"secondary_y": True}]]))
    
    
    
    for col in data.columns:
        if("Dividend" in col):
            fig.add_trace(
                go.Scattergl(x=list(df[df[col]>0].index), y=list(df[df[col]>0][col]),
                name = col, marker=dict(color="green", size=2), line = dict(color='grey', width=1, dash='dash')),
                row=1, col=1,
                secondary_y=True
            )
        else:
            fig.add_trace(
                go.Scattergl(x=list(df.index), y=list(df[col]),
                name = col),
                row=1, col=1
            )
            
    
    df["const"] = 0
    
    fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df["const"]), line = dict(color='grey', width=1, dash='dash'),
            name = "zero"),
            row=1, col=1
    )
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig['layout'].update(height=height, width=width) 
    return fig

In [201]:
def main(stocks, start_date = datetime.now() - timedelta(days=365*3), end_date = datetime.now(),
              cols = ['Spent', 'Gross',  'Dividend', 'Close', 'ClosePct','NetPct']):
    data = None
    columns = []
    for stock in stocks:
        print("Working on {0} ...".format(stock))
        if(len(stocks)>1):
            for col in cols:
                columns.append(stock+'_'+col)
        
        init_amt = 10000
        fee = 0.01  #1%
        monthly_topup = 0
        price_data = get_price_data(stock, start_date, end_date)
        dividend_data = get_dividend_data(stock, start_date, end_date)
        returns = calculate_profit(price_data, dividend_data, init_amt, fee, monthly_topup)
        returns = returns.set_index('Date')
        if(len(stocks)>1):
            returns = returns.add_prefix(stock+'_')
        if(data is None):
            data = returns
        else:
            data = data.join(returns)
    if(len(stocks)==1):
        columns = cols
    return plot_chart(data[columns], 500, 900)

In [206]:
#plot_stock("O39.SI", datetime(2017,1,1),  datetime(2021,1,1))
#cols = ['Spent', 'Gross',  'Dividend', 'Close', 'ClosePct','NetPct']
main(stocks = ["O39.SI", "D05.SI", "U11.SI"], 
     start_date = datetime.now() - timedelta(days=365*2),  
     #start_date = datetime(2017,1,1),  
     #end_date = datetime(2021,1,1), 
     cols = ["ClosePct", "NetPct"]
     #cols = ['Spent', 'Gross',  'Dividend', 'Close', 'ClosePct','NetPct']
    )



Working on O39.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...


/home/ysj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Working on D05.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...
Working on U11.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...
Ploting chart ...


FigureWidget({
    'data': [{'name': 'O39.SI_ClosePct',
              'type': 'scattergl',
              'uid'…

In [ ]:
'''
Frasers L&I Tr (BUOU / FRAE.SI)	BUOU.SI
ManulifeReit USD (BTOU / MANU.SI)	BTOU.SI
Frasers Hospitality Trust Stapled Securities (ACV.SI)	ACV.SI

Mapletree NAC Tr (RW0U / MAPE.SI)	RW0U.SI
Ascendas REIT	A17U.SI
SPHREIT (SK6U / SPHR.SI)	SK6U.SI
AIMS APAC REIT (O5RU.SI)	O5RU.SI
CapitaR China Tr (AU8U / CRCT.SI)	AU8U.SI
DBS (D05 / DBSM.SI)	D05.SI

OCBC Bank (O39 / OCBC.SI)	O39.SI
Suntec Reit T82U SUNT.SI	T82U.SI
Sasseur Reit (CRPU / SASS.SI)	CRPU.SI
Ascott Real Estate Investment Trust (HMN.SI)	HMN.SI
OUE Commercial Real Estate Investment Trust (TS0U.SI)	TS0U.SI
'''
main(stocks = ["A17U.SI", "BTOU.SI", "SK6U.SI", "ACV.SI", "AU8U.SI","BUOU.SI","RW0U.SI","O5RU.SI","T82U.SI","CRPU.SI","HMN.SI","TS0U.SI"], 
     start_date = datetime.now() - timedelta(days=365*5),  
     #start_date = datetime(2017,1,1),  
     #end_date = datetime(2021,1,1), 
     cols = ["ClosePct", "NetPct"]
     #cols = ['Spent', 'Gross',  'Dividend', 'Close', 'ClosePct','NetPct']
    )



Working on A17U.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...


/home/ysj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Working on BTOU.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...
Working on SK6U.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...
Working on ACV.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...
Working on AU8U.SI ...
Getting price data ...
Getting dividend data ...
Calculating returns ...
